**Convolutional Neural Network (CNN) basierend auf die YOLO-Architektur zur Erkennung von deutschen Verkehrszeichen in Echtzeit mit dem GTSRB-Dataset**

Autoren: Laieb Ahmad, Enes Bayar, Mehmet Akif Onur & Mert-Can Öksuz
<br>
Datum: 19.08.2024
<br>
Modul: KI-DL/ML

## **Befehle zum Installieren/Upgraden von Packages** 
Hiert führen wir die Installation (und nach Bedarf ein Upgrade) wichtiger Python-Pakete durch, die für unser Projekt benötigt werden. Wir verwenden die Bash-Magie `%%bash`, um mehrere `pip install`-Befehle in einer einzigen Zelle auszuführen. Dadurch wird die Effizienz erhöht und der Code übersichtlicher gestaltet. 

Die folgenden Pakete werden aktualisiert:
- **Numpy**: Eine Bibliothek für numerische Berechnungen und Unterstützung von Arrays.
- **Pandas**: Eine leistungsstarke Bibliothek zur Datenmanipulation und -analyse.
- **Scikit-learn**: Ein Framework für maschinelles Lernen mit zahlreichen Algorithmen und Werkzeugen.
- **Scikit-image**: Eine Sammlung von Algorithmen zur Bildverarbeitung.
- **Pillow**: Eine Bibliothek zur Bildbearbeitung und -manipulation.
- **Imbalanced-learn**: Eine Bibliothek zur Behandlung von unausgewogenen Datensätzen im maschinellen Lernen.
- **Statsmodels**: Eine Bibliothek für statistische Modelle und Hypothesentests.
- **Matplotlib**: Ein Plotting-Toolkit zur Datenvisualisierung.
- **Seaborn**: Eine auf Matplotlib basierende Bibliothek zur statistischen Datenvisualisierung.
- **TensorFlow**: Ein Framework für maschinelles Lernen und Deep Learning.
- **Ultralytics**: Eine Bibliothek, die das YOLO-Framework für die Objekt- und Verkehrszeichenerkennung bereitstellt.

Durch die Verwendung dieser Bash-Zelle stellen wir sicher, dass alle erforderlichen Pakete auf dem neuesten Stand sind, um optimale Leistung und Kompatibilität während der Entwicklung zu gewährleisten.


In [ ]:
%%bash #Anstatt einzel % vor jedem Command, gesammelt in einer Zelle
pip install numpy --upgrade
pip install pandas --upgrade
pip install scikit-learn --upgrade
pip install scikit-image --upgrade
pip install pillow --upgrade
pip install imbalanced-learn --upgrade
pip install statsmodels --upgrade
pip install matplotlib --upgrade
pip install seaborn --upgrade
pip install tensorflow --upgrade
pip install ultralytics --upgrade#

## **Laden von Packages** 
In diesem Abschnitt importieren wir die erforderlichen Bibliotheken, die für die Entwicklung und Ausführung unseres YOLOv5-Projekts notwendig sind. 

- **`from ultralytics import YOLO`**: Importiert das YOLO-Modul von Ultralytics, das für die Objekt- und Verkehrszeichenerkennung verwendet wird.
- **`import os`**: Ermöglicht den Zugriff auf Funktionen zur Interaktion mit dem Betriebssystem, wie das Verwalten von Dateipfaden und Verzeichnissen.
- **`import numpy as np`**: Importiert die Numpy-Bibliothek, die grundlegende Funktionen für numerische Berechnungen und Arrays bereitstellt.
- **`import pandas as pd`**: Importiert Pandas, das für die Datenmanipulation und -analyse verwendet wird, insbesondere für die Arbeit mit DataFrames.
- **`import cv2`**: Importiert OpenCV, eine Bibliothek für Computer Vision, die Funktionen zur Bild- und Videoverarbeitung bereitstellt.
- **`import shutil`**: Ermöglicht die Durchführung von Datei- und Verzeichnisoperationen, wie das Kopieren und Verschieben von Dateien.
- **`import matplotlib`**: Importiert Matplotlib, eine Bibliothek zur Datenvisualisierung, die häufig zur Erstellung von Diagrammen und Grafiken verwendet wird.
- **`import csv`**: Importiert die CSV-Bibliothek, die das Arbeiten mit CSV-Dateien zur Datenspeicherung und -verarbeitung erleichtert.
- **`import torch`**: Importiert PyTorch, ein Framework für maschinelles Lernen, das für die Implementierung und das Training von Deep-Learning-Modellen verwendet wird.

Durch diese Importe stellen wir sicher, dass wir alle notwendigen Funktionen und Werkzeuge zur Verfügung haben, um unser Projekt erfolgreich umzusetzen.


In [1]:
from ultralytics import YOLO
from helfer_funktionen import unzip_file, resize_images, convert_labels_to_yolo_format, split_train_val
import os
import numpy as np
import pandas as pd
import cv2
import shutil
import matplotlib
import csv
import torch

## **Übersicht der Versionen von den genutzten Packages** 
In diesem Abschnitt überprüfen wir die installierten Versionen wichtiger Pakete, die in unserem Projekt verwendet werden. Wir nutzen `print()`-Befehle, um die Versionen der Pakete Numpy, Pandas, OpenCV, Matplotlib, CSV und PyTorch anzuzeigen. Dies ermöglicht uns, sicherzustellen, dass die richtigen Versionen installiert sind und die Kompatibilität zwischen den Bibliotheken gegeben ist. 


In [14]:
print("Package 'Numpy':",np.__version__)
print("Package 'Pandas':",pd.__version__)
print("Package 'OpenCV':",cv2.__version__)
print("Package 'Matplotlib':",matplotlib.__version__)
print("Package 'CSV':",csv.__version__)
print("Package 'PyTorch':",torch.__version__)

Package 'Numpy': 1.26.4
Package 'Pandas': 2.2.3
Package 'OpenCV': 4.10.0
Package 'Matplotlib': 3.9.2
Package 'CSV': 1.0
Package 'PyTorch': 2.4.1+cpu


## **Installieren von Yolov5** 
In diesem Abschnitt klonen wir das YOLOv5-Repository von GitHub, um auf den Quellcode und die Modelle zuzugreifen. Zuerst verwenden wir den Befehl `!git clone https://github.com/ultralytics/yolov5`, um das Repository auf unser lokales System herunterzuladen. Anschließend wechseln wir mit `%cd yolov5` in das Verzeichnis des geklonten Projekts, um dort weitere Schritte auszuführen. Schließlich installieren wir mit `%pip install -r requirements.txt` alle erforderlichen Abhängigkeiten, die für den Betrieb des YOLOv5-Modells notwendig sind.


In [ ]:
!git clone https://github.com/ultralytics/yolov5

%%bash
cd yolov5
pip install -r requirements.txt

## **Entpacken der .zip-Dateien des GTSRB-Datensatzes** 
Eine Übersicht über die Versionen der einzelnen Packages/Frameworks. Das dient dazu, die Kompitabilität der Packages mit der Python-Version (oder mit der miniconda-Version) zu überwachen.

In [ ]:
# Pfade zu den heruntergeladenen Dateien
train_images_zip = 'Pfad_zur_GTSRB_Final_Training_Images.zip'
test_images_zip = 'Pfad_zur_GTSRB_Final_Test_Images.zip'
test_labels_zip = 'Pfad_zur_GTSRB_Final_Test_GT.zip'

# Entpacken der Dateien
unzip_file(train_images_zip, 'GTSRB_Training')
unzip_file(test_images_zip, 'GTSRB_Test')
unzip_file(test_labels_zip, 'GTSRB_Test_GT')

## **Importieren des Datensatzes** 
Eine Übersicht über die Versionen der einzelnen Packages/Frameworks. Das dient dazu, die Kompitabilität der Packages mit der Python-Version (oder mit der miniconda-Version) zu überwachen.

In [ ]:
# Beispiel-Code 
%%bash 
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/kaggle.json
chmod 600 ~/.kaggle/kaggle.json
pip install kaggle
kaggle datasets download -d andrewmvd/road-sign-detection
unzip road-sign-detection.zip -d data
mv ./data/annotations ./data/labels

## **Übersicht über die Daten** 
Eine Übersicht über die Versionen der einzelnen Packages/Frameworks. Das dient dazu, die Kompitabilität der Packages mit der Python-Version (oder mit der miniconda-Version) zu überwachen.

## **Splitten des Datensatzes in Train und Validation** 


AttributeError: module 'csv' has no attribute 'file'

## **Erstellen der yaml-Datei in dem Projektverzeichnis** 
